In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#!pip install scikit-image
from skimage import io
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import nltk
from nltk.util import ngrams
from spacy.lang.en import English
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
nlp = English()

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

In [ ]:
!pip install -q xlrd
!git clone https://github.com/ana-bca/Proyecto-Mineria-de-Datos

Cloning into 'Proyecto-Mineria-de-Datos'...
remote: Enumerating objects: 68487, done.
remote: Total 68487 (delta 0), reused 0 (delta 0), pack-reused 68487
Receiving objects: 100% (68487/68487), 1.98 GiB | 36.14 MiB/s, done.
Resolving deltas: 100% (905/905), done.
Checking out files: 100% (38664/38664), done.


In [ ]:
# Lectura de tablas ==========================================================#
#data_F = pd.read_csv('Proyecto-Mineria-de-Datos/data_F.csv', engine='python',sep=",")
data_train = pd.read_csv('Proyecto-Mineria-de-Datos/Imagenes/data_train.csv', engine='python',sep=",")
data_test = pd.read_csv('Proyecto-Mineria-de-Datos/Imagenes/data_test.csv', engine='python',sep=",")
data_valid = pd.read_csv('Proyecto-Mineria-de-Datos/Imagenes/data_valid.csv', engine='python',sep=",")

Ngrams_1 = pd.read_csv('Proyecto-Mineria-de-Datos/palabras_finales.csv', engine='python',sep="|")
Ngrams_2 = pd.read_csv('Proyecto-Mineria-de-Datos/prueba1.csv',sep=",")
# Funciones ==================================================================#
N2 = Ngrams_2.compuesta.values
def TUPT(x):
    x_N1 = np.array(NGRAM(x,1))
    x_N1 = x_N1.reshape(x_N1.shape[0],)
    #x_N2 = pd.DataFrame(np.array(NGRAM(data_F.caption[0],2)))
    #x_N2 = x_N2.agg(' '.join, axis=1).values
    #x_N2 = np.array([x for x in x_N2 if x in N2])
    #x_T = list(np.append(x_N1.astype(str), x_N2.astype(str)))
    x_T = list(set(x_N1.astype(str)))
    return x_T
def NGRAM(x,n):
    token=nltk.word_tokenize(str(x))
    bigrams=ngrams(token,n)
    return list(bigrams)

In [ ]:
data_train.head(5)

,photo_id,caption
0,D3Bdkrbl9dsW_fCFChEpXQ.jpg,black curry steamed rice
1,qrdZ0Fcvf-BvHRxydPF8SA.jpg,chicken
2,fw7t-SROxqh5t1QtoxHX0g.jpg,pork chashu rice
3,B1Bd1cThPSE-hWqKYhbzjg.jpg,flavorful chicken
4,zNXm1O4RGLq0tRV9-6TVsg.jpg,strawberry wrap


In [ ]:
# Generamos las clases en los dataframe
import nltk
nltk.download('punkt')
data_train = data_train.assign(tags = np.array(data_train.caption.map(lambda p: TUPT(p))))
data_test = data_test.assign(tags = np.array(data_test.caption.map(lambda p: TUPT(p))))
data_valid = data_valid.assign(tags = np.array(data_valid.caption.map(lambda p: TUPT(p))))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data_train.head()

,photo_id,caption,tags
0,D3Bdkrbl9dsW_fCFChEpXQ.jpg,black curry steamed rice,"[curry, steamed, rice, black]"
1,qrdZ0Fcvf-BvHRxydPF8SA.jpg,chicken,[chicken]
2,fw7t-SROxqh5t1QtoxHX0g.jpg,pork chashu rice,"[rice, pork, chashu]"
3,B1Bd1cThPSE-hWqKYhbzjg.jpg,flavorful chicken,"[chicken, flavorful]"
4,zNXm1O4RGLq0tRV9-6TVsg.jpg,strawberry wrap,"[wrap, strawberry]"


In [ ]:
# Cree un ImageDataGenerator con flow_from_dataframe
## Clases en los datos
Ngrams_1 = Ngrams_1.word.values
#Ngrams_2 = Ngrams_2.compuesta[Ngrams_2.Clase == 1].values
#keys =   np.append(Ngrams_1, Ngrams_2) # Estas son las clases definitivas
keys = list(set(Ngrams_1.astype(str)))

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe = data_train[["photo_id","tags"]].sample(5000),
    directory = "Proyecto-Mineria-de-Datos/Imagenes/train",
    x_col = "photo_id",
    y_col = "tags",
    batch_size = 32,
    class_mode = "categorical",
    color_mode = "rgb",
    classes = keys,
    target_size = (400,500))

valid_generator = datagen.flow_from_dataframe(
    dataframe = data_valid[["photo_id","tags"]].sample(1000),
    directory = "Proyecto-Mineria-de-Datos/Imagenes/valid",
    x_col = "photo_id",
    y_col = "tags",
    batch_size = 32,
    class_mode = "categorical",
    color_mode = "rgb",
    classes = keys,
    target_size = (400,500))

test_generator = datagen.flow_from_dataframe(
    dataframe = data_test[["photo_id","tags"]].sample(1000),
    directory = "Proyecto-Mineria-de-Datos/Imagenes/test",
    x_col = "photo_id",
    y_col = "tags",
    batch_size = 32,
    class_mode = None,
    color_mode = "rgb",
    target_size = (400,500))

Found 5000 validated image filenames belonging to 983 classes.
Found 1000 validated image filenames belonging to 983 classes.
Found 1000 validated image filenames.


In [ ]:
# Construye el modelo:
model = Sequential()
model.add(Conv2D(32,(3,3),padding = 'same',input_shape = (400,500,3))) 
model.add(Activation ('relu'))
model.add(Conv2D(32,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),padding = 'same')) 
model.add(Activation('relu')) 
model.add(Conv2D(64,(3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size = (2,2))) 
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dropout (0.5))
model.add(Dense(983,activation='sigmoid'))
model.compile(optimizers.rmsprop(lr = 0.0001,decay = 1e-6),loss = "binary_crossentropy", metrics = ["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3
)

Epoch 1/3
156/156 [==============================] - 4668s 30s/step - loss: 0.3258 - accuracy: 0.8462 - val_loss: 0.0569 - val_accuracy: 0.9971
Epoch 2/3
156/156 [==============================] - 4716s 30s/step - loss: 0.0428 - accuracy: 0.9904 - val_loss: 0.0332 - val_accuracy: 0.9972
Epoch 3/3
 87/156 [===============>..............] - ETA: 33:00 - loss: 0.0256 - accuracy: 0.9966

In [ ]:
# Graficas de validaciones ====================================================#
plt.figure(0)  
plt.plot(history.history['acc'],'r')  
plt.plot(history.history['val_acc'],'g')  
plt.xticks(np.arange(0, 20, 1.0))  
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Accuracy")  
plt.title("Training Accuracy vs Validation Accuracy")  
plt.legend(['train','validation'])
plt.figure(1)  
plt.plot(history.history['loss'],'r')  
plt.plot(history.history['val_loss'],'g')  
plt.xticks(np.arange(0, 20, 1.0))  
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Loss")  
plt.title("Training Loss vs Validation Loss")  
plt.legend(['train','validation'])
plt.show()  

In [ ]:
loss, accuracy = model.evaluate_generator(validation_generator, nb_validation_samples/batch_size, workers=12)
print("Exit Accuracy:  {:.4f}".format(accuracy))